In [1]:
!wget https://raw.githubusercontent.com/SirYoja/TaskData/refs/heads/main/Neo-Tokio/test.csv -O test.csv
!wget https://raw.githubusercontent.com/SirYoja/TaskData/refs/heads/main/Neo-Tokio/train.csv -O train.csv

--2025-04-29 21:37:45--  https://raw.githubusercontent.com/SirYoja/TaskData/refs/heads/main/Neo-Tokio/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50420 (49K) [text/plain]
Saving to: ‘test.csv’

test.csv            100%[===================>]  49.24K  --.-KB/s    in 0.009s  

2025-04-29 21:37:46 (5.59 MB/s) - ‘test.csv’ saved [50420/50420]

--2025-04-29 21:37:46--  https://raw.githubusercontent.com/SirYoja/TaskData/refs/heads/main/Neo-Tokio/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153226 (150K) [text/pl

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 5.2 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, Dataset, Subset
import torch.nn.functional as F
from tqdm.notebook import tqdm
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
class DatasetNeoTokio(Dataset):
  def __init__(self, path, train='True'):
    data=pd.read_csv(path)
    self.train=train
    if self.train=='True':
      self.label=torch.tensor((data['target'].values), dtype=torch.uint8)
      self.features=torch.tensor((data.drop('target', axis=1).values), dtype=torch.float32)
    else:
      self.features=torch.tensor(data.values, dtype=torch.float32)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    if self.train=='True':
      return self.features[idx], self.label[idx]
    else:
      return self.features[idx]

full_dataset=DatasetNeoTokio('./train.csv')
train_idxs, val_idxs=train_test_split(range(len(full_dataset)), test_size=0.1, train_size=0.9)
train_split=Subset(full_dataset, train_idxs)
val_split=Subset(full_dataset, val_idxs)

test=DatasetNeoTokio('./test.csv', train='False')




In [ ]:
train_data_loader=DataLoader(train_split, batch_size=12900, shuffle=True)
val_data_loader=DataLoader(val_split, batch_size=12900, shuffle=True)

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler as scr

In [ ]:
for dat, label in train_data_loader:
  datat=np.array(dat)
  labelst=np.array(label)


for dat, label in val_data_loader:
  datav=np.array(dat)
  labelsv=np.array(label)

datats=np.array(test)

sc=scr()

sc.fit(datat)
datat=sc.transform(datat)
datav=sc.transform(datav)
datats=sc.transform(datats)


In [ ]:
cls=CatBoostClassifier(iterations=300, depth=3)
cls.fit(datat, labelst)

Learning rate set to 0.033865
0:	learn: 0.6657773	total: 53.6ms	remaining: 16s
1:	learn: 0.6429366	total: 56.6ms	remaining: 8.43s
2:	learn: 0.6207912	total: 60.5ms	remaining: 5.99s
3:	learn: 0.6017887	total: 68.5ms	remaining: 5.07s
4:	learn: 0.5809693	total: 76.4ms	remaining: 4.51s
5:	learn: 0.5621782	total: 96.6ms	remaining: 4.74s
6:	learn: 0.5450513	total: 108ms	remaining: 4.54s
7:	learn: 0.5276842	total: 114ms	remaining: 4.18s
8:	learn: 0.5127654	total: 125ms	remaining: 4.05s
9:	learn: 0.5011719	total: 133ms	remaining: 3.86s
10:	learn: 0.4891661	total: 145ms	remaining: 3.81s
11:	learn: 0.4787743	total: 151ms	remaining: 3.62s
12:	learn: 0.4692168	total: 163ms	remaining: 3.6s
13:	learn: 0.4585969	total: 170ms	remaining: 3.48s
14:	learn: 0.4486909	total: 186ms	remaining: 3.53s
15:	learn: 0.4402980	total: 198ms	remaining: 3.52s
16:	learn: 0.4325691	total: 203ms	remaining: 3.38s
17:	learn: 0.4261850	total: 211ms	remaining: 3.3s
18:	learn: 0.4198190	total: 223ms	remaining: 3.3s
19:	learn:

In [ ]:
val=cls.predict(datav)
print(accuracy_score(labelsv, val))

0.8615384615384616


In [ ]:
test1=cls.predict(datats)
f=open('answers.csv', 'w')
for i in test1:
  f.write(str(i))
  f.write('\n')
f.close()